In [48]:
import numpy as np
import os
import glog as log
import copy

from __future__ import division

import open3d as o3d
from open3d import JVisualizer
import pandas as pd

from evaluation.tools.mesh import Mesh
from evaluation.tools.mesh_evaluator import MeshEvaluator

# Rotation matrices:
# East North Up (ENU) frame to Unity's world frame of reference
enu_R_unity = np.array([[1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]])
unity_R_enu = np.transpose(enu_R_unity)

# Right Handed frame to Unity's Left Handed frame of reference
righthand_R_lefthand = np.array([[1, 0, 0],
                                 [0, -1, 0],
                                 [0, 0, 1]])
lefthand_R_righthand = np.transpose(righthand_R_lefthand)

In [49]:
# FILL PATHS BELOW
#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Downloads/tesse_semantics_2.ply"

#gt_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"

#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/tesse_semantics_3.ply"

gt_mesh_path = "/home/tonirv/Downloads/office1_tony.ply"

est_base_path = "/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/"
est_mesh_names = ["humans_6_long_gt_dyn.ply",
                  "humans_6_long_gt_no_dyn.ply",
                  "humans_12_long_gt_dyn.ply",
                  "humans_12_long_gt_no_dyn.ply",
                  "humans_30_long_gt_dyn.ply",
                  "humans_30_long_gt_no_dyn.ply"
                 ]

est_mesh_paths = []
for est_mesh_name in est_mesh_names:
    est_mesh_path = est_base_path + est_mesh_name
    est_mesh_paths.append(est_mesh_path)
    print(est_mesh_path)


#est_mesh_path = "/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_6_long_gt_no_dyn.ply"

#est_mesh_path = "/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_12_long_gt_dyn.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_12_long_gt_no_dyn.ply"

#est_mesh_path = "/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_12_long_gt_dyn.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_12_long_gt_no_dyn.ply"

/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_6_long_gt_dyn.ply
/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_6_long_gt_no_dyn.ply
/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_12_long_gt_dyn.ply
/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_12_long_gt_no_dyn.ply
/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_30_long_gt_dyn.ply
/home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_30_long_gt_no_dyn.ply


In [50]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)

# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)

# Align Pointclouds Manually:
#est_mesh.mesh_o3d.translate([0, -5, 0])
#gt_mesh.transform_left(righthand_R_lefthand)
gt_mesh.transform_left(enu_R_unity)

vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(est_mesh.mesh_o3d)
vis.add_geometry(gt_mesh.mesh_o3d)
vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
vis.run()
vis.destroy_window()

NUMBER_OF_SAMPLES=1000000
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, NUMBER_OF_SAMPLES)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 1572372 points and 524124 triangles.
[[14.5       4.       27.5     ]
 [13.5       4.       39.5     ]
 [14.5       4.       39.5     ]
 ...
 [ 6.567781  2.118583 36.00596 ]
 [ 6.568742  2.126512 36.21212 ]
 [ 6.568742  2.126512 36.00596 ]]
[[      0       1       2]
 [      3       4       5]
 [      6       7       8]
 ...
 [1572363 1572364 1572365]
 [1572366 1572367 1572368]
 [1572369 1572370 1572371]]

Transforming mesh according to left matrix:
[[1 0 0]
 [0 0 1]
 [0 1 0]]


In [57]:
print("Loading Estimated meshes...")
dict_est_pcls = dict()
for est_mesh_path in est_mesh_paths:
    est_mesh_original = Mesh(est_mesh_path)
    est_mesh = copy.deepcopy(est_mesh_original)
    # Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
    # est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
    est_pcl = o3d.io.read_point_cloud(est_mesh_path)
    dict_est_pcls[est_mesh_path] = (est_pcl)

Loading Estimated meshes...
Testing mesh in open3d ...
geometry::TriangleMesh with 5691863 points and 6674991 triangles.
[[ 28.365    25.485     1.02251]
 [ 28.335    25.455     1.01616]
 [ 28.365    25.455     1.01376]
 ...
 [-12.045    23.535     3.96728]
 [-12.045    23.5224    3.975  ]
 [-12.075    23.507     4.005  ]]
[[      0       1       2]
 [      3       1       0]
 [      4       3       0]
 ...
 [5691814 5691817 5691812]
 [5691862 5691725 5691724]
 [5691725 5691862 5691758]]

Testing mesh in open3d ...
geometry::TriangleMesh with 5581641 points and 6521821 triangles.
[[27.885   24.525    1.00789]
 [27.855   24.495    1.00704]
 [27.885   24.495    1.0068 ]
 ...
 [16.7972  28.605    1.875  ]
 [16.7864  28.575    1.875  ]
 [16.785   28.575    1.87596]]
[[      0       1       2]
 [      3       1       0]
 [      4       3       0]
 ...
 [5581634 5581635 5581636]
 [5581637 5581638 5581639]
 [5581637 5581639 5581640]]

Testing mesh in open3d ...
geometry::TriangleMesh with 566

In [52]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [53]:
# ICP params
ICP_THRESHOLD = 1.5
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [54]:
def visualize_two_pcls(gt_pcl, est_pcl):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.get_render_option().mesh_show_back_face = True
    vis.add_geometry(gt_pcl)
    vis.add_geometry(est_pcl)
    vis.run()
    vis.destroy_window()  

In [62]:
visualize = False
def evaluate_icp_for_clouds(gt_pcl, dict_est_pcls):
    for est_pcl_key in dict_est_pcls:
        est_pcl = dict_est_pcls[est_pcl_key]
        if (visualize):
            visualize_two_pcls(gt_pcl, est_pcl)
            draw_registration_result(est_pcl, gt_pcl, trans_init)
            
        evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)
        reg_p2p = o3d.registration.registration_icp(
        est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
        o3d.registration.TransformationEstimationPointToPoint(),
        o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
        correspondences = reg_p2p.correspondence_set
        
        if (visualize):
            # Draw Registration Result
            draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)
        
        print("# # # # REGISTRATION INLIER RMSE for: %s " % est_pcl_key)
        print(reg_p2p.inlier_rmse)
        print("")


In [63]:
# RUN FULL EVALUATION
evaluate_icp_for_clouds(gt_pcl, dict_est_pcls)

# # # # REGISTRATION INLIER RMSE for: /home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_12_long_gt_no_dyn.ply 
0.133386353341

# # # # REGISTRATION INLIER RMSE for: /home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_30_long_gt_dyn.ply 
0.0610379666296

# # # # REGISTRATION INLIER RMSE for: /home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_12_long_gt_dyn.ply 
0.133685119516

# # # # REGISTRATION INLIER RMSE for: /home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_6_long_gt_no_dyn.ply 
0.0892965288129

# # # # REGISTRATION INLIER RMSE for: /home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_30_long_gt_no_dyn.ply 
0.192013593816

# # # # REGISTRATION INLIER RMSE for: /home/tonirv/Code/ROS/kimera_ws/src/Kimera-Semantics/kimera_semantics_ros/mesh_results/humans_6_long_gt_dyn.ply 

In [42]:
# Visualize initial registration problem
if (visualize):
    draw_registration_result(est_pcl, gt_pcl, trans_init)

In [13]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [14]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.060733, and correspondence_set size of 5691863
Access transformation to get result.


In [15]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [24]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

print("# # # # REGISTRATION INLIER RMSE: # # # # ")
print(reg_p2p.inlier_rmse)
print("")

registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.060622, and correspondence_set size of 5691863
Access transformation to get result.

Transformation is:
[[ 9.99999989e-01 -1.86947698e-06 -1.48213985e-04  1.57051234e-03]
 [ 1.86118979e-06  9.99999998e-01 -5.59138522e-05 -1.04631268e-03]
 [ 1.48214089e-04  5.59135757e-05  9.99999987e-01 -5.94344733e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 5691863 elements.
Use numpy.asarray() to access data.

# # # # REGISTRATION INLIER RMSE: # # # # 
0.0606218589518

